# CTA200H Assignment 3 - Ian Niebres

In [1]:
import numpy as np
from iteration import iterate
import matplotlib.pyplot as plt

In [5]:
%%time
N = 5
z0 = 0 

x_arr = np.arange(-2, 2, 1)
y_arr = np.arange(-2, 2, 1)

z_arr = []

for x in x_arr:
    for y in y_arr:
        z_arr.append([iterate(z0, N, x, y)])

CPU times: user 739 µs, sys: 79 µs, total: 818 µs
Wall time: 792 µs


In [ ]:
for i in range(len(z_arr)):
    plt.plot(z_arr[i][0].real, z_arr[i][0].imag, 'r' if z**2)

In [7]:
z_arr[0]

[(array([  0.   +0.j,  -2.   -2.j,  -2.   +6.j, -34.  -26.j, 478.+1766.j]),
  array([   0.,    8.,   40., 1832.,    0.]))]

In [ ]:
iterate(z0, 2, 1, 1)